In [1]:
from transformer_lens import HookedTransformer
import torch as t

In [3]:
gpt2 = HookedTransformer.from_pretrained('gpt2-small')

Loaded pretrained model gpt2-small into HookedTransformer


[17250, 616, 1438, 318]
[2061, 466, 345, 892]


In [32]:
gpt2 = HookedTransformer.from_pretrained('gpt2-small')

prompt1 = 'Hi my name is'
prompt2 = 'What do you think'
print(gpt2.tokenizer.encode(prompt1))
print(gpt2.tokenizer.encode(prompt2))

logits1 = gpt2(prompt1)
logits2 = gpt2(prompt2)
logits1.retain_grad()
logits2.retain_grad()

loss1 = (logits1**2).sum()
loss2 = (logits2**2).sum()
loss1.backward(retain_graph=True)
loss2.backward(retain_graph=True)
grad1 = logits1.grad.clone()
grad2 = logits2.grad.clone()
print(grad1)
print(grad2)

#now do the cloning thing
out = logits1.clone()
out.retain_grad()
out[:,:,2:] = logits2[:,:,2:]
loss = (out**2).sum()
loss.backward()
grad = out.grad
print(grad)

assert t.allclose(grad[:,:,:2], grad1[:,:,:2])
assert t.allclose(grad[:,:,2:], grad2[:,:,2:])

tensor([[[ 15.0523,  22.2428,  15.7838,  ...,  -6.2597,  -6.7747,  17.1868],
         [ 25.3996,  10.7595,   3.9764,  ...,  -2.4713,  -2.2165,  11.1564],
         [ 13.0060,   9.3690,   7.8577,  ...,   3.3197,  -4.1594,   6.5407],
         [ 16.8229,  15.6519,   5.6569,  ...,  -3.0447, -10.3274,   9.1122],
         [  9.9818,   9.3194,   2.3506,  ...,  -5.0371,  -4.7460,   9.1493]]],
       device='cuda:0')
tensor([[[15.0523, 22.2428, 15.7838,  ..., -6.2597, -6.7747, 17.1868],
         [12.7488, 10.6369,  5.5582,  ...,  1.0105, -4.8911, 11.3958],
         [ 7.4847,  6.7689,  2.5595,  ...,  3.4052, -4.3795,  9.4039],
         [17.3336, 14.7194,  8.1185,  ...,  1.2838, -2.7686, 12.9945],
         [20.0961, 14.0792,  6.6520,  ..., -0.8238, -1.4198, 16.7414]]],
       device='cuda:0')
tensor([[[15.0523, 22.2428, 15.7838,  ..., -6.2597, -6.7747, 17.1868],
         [25.3996, 10.7595,  5.5582,  ...,  1.0105, -4.8911, 11.3958],
         [13.0060,  9.3690,  2.5595,  ...,  3.4052, -4.3795,  9.40

In [12]:
logits1[:,:,5:] = logits2[:,:,5:]